# Notebook to classify power plants
Variables include:
- Fuel type
- Historical ramp speeds
- Historical efficiency
- Location?

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/Home/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Variable definitions

These variables will be used to cluster the power plants.
- year: year that the data is for
- fuel type: primary fuel type used at the facility (SUB, LIG, NG, etc)
- 1-hr ramp rate: maximum or 95 percentile rate of generation increase over 1 hour
- 3-hr ramp rate: maximum or 95 percentile rate of generation increase over 3 hours
- efficiency: might just change this to heat rate (MMBTU/MWh)
- efficiency std: standard deviation in efficiency based on monthly values for the year
- CF: capacity factor, which is the amount generated in a year divided by the theoretical maximum generation possible $MWh/(Capacity * 8760)$ (or hours per month for monthly values used in calculating the standard deviation)
- CF std: standard deviation of CF based on monthly values for the year

In [3]:
cols = ['year', 'fuel type', '1-hr ramp rate', '3-hr ramp rate',
        'efficiency', 'efficiency std', 'CF', 'CF std']

#Add index of power plant IDs
df = pd.DataFrame(columns=cols)

In [4]:
df

,year,fuel type,1-hr ramp rate,3-hr ramp rate,efficiency,efficiency std,CF,CF std
